# Manually Scrape a Real Estate Webpage

In [ ]:
!pip install selenium
#import libraries for scraping
import requests
from bs4 import BeautifulSoup
import selenium


In [5]:
#requesting the webpage
l=[]
o={}
target_url = "https://www.redfin.com/CA/Livermore/La-Vina-Apartments/apartment/177431043"  #random rental listing
head={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"}
resp = requests.get(target_url, headers=head)

print(resp.status_code)

200


In [ ]:
#parse the webpage
soup = BeautifulSoup(resp.content, 'html.parser')
soup.prettify()

In [7]:
#find the price of property 
prices = soup.find_all('div', class_='statsValue price')
for price in prices:
    print(price.get_text())


$2,840+ /mo


In [8]:
#find the address of property 
addresses = soup.find_all('div', class_='street-address')
for address in addresses:
    print(address.get_text())

4601 Gerrilyn Way, Livermore, CA 94550


In [9]:
#find property attributes
attribute = soup.find_all('div', class_='secondary-stats')
for attr in attribute:
    print(attr.get_text())


2bd•2ba•887sq ft


In [ ]:
image = soup.find_all('img')
for img in image:
    print(img['src'])



# Foxit API to convert created HTML Pages to PDF

In [62]:
import os
import requests
import sys 
from time import sleep 

base_url = "https://na1.fusion.foxit.com"
client_id = "foxit_QXn-i7EcDW1Uvgdh"
client_secret = "EebeG3bXf3XMui-9P0BA9oTiXxjs-dZs"

HOST = base_url
CLIENT_ID = client_id
CLIENT_SECRET = client_secret

def uploadDoc(path, id, secret):
	
	headers = {
		"client_id":id,
		"client_secret":secret
	}

	with open(path, 'rb') as f:
		files = {'file': f}

		request = requests.post(f"{HOST}/pdf-services/api/documents/upload", files=files, headers=headers)
		return request.json()


def htmlToPDF(doc, id, secret):
	
	headers = {
		"client_id":id,
		"client_secret":secret,
		"Content-Type":"application/json"
	}

	body = {
		"documentId":doc	
	}

	request = requests.post(f"{HOST}/pdf-services/api/documents/create/pdf-from-html", json=body, headers=headers)
	return request.json()

def checkTask(task, id, secret):

	headers = {
		"client_id":id,
		"client_secret":secret,
		"Content-Type":"application/json"
	}

	done = False
	while done is False:

		request = requests.get(f"{HOST}/pdf-services/api/tasks/{task}", headers=headers)
		status = request.json()
		if status["status"] == "COMPLETED":
			done = True
			return status
		elif status["status"] == "FAILED":
			print("Failure. Here is the last status:")
			print(status)
			sys.exit()
		else:
			print(f"Current status, {status['status']}, percentage: {status['progress']}")
			sleep(5)


def downloadResult(doc, path, id, secret):
	
	headers = {
		"client_id":id,
		"client_secret":secret
	}

	with open(path, "wb") as output:
		
		bits = requests.get(f"{HOST}/pdf-services/api/documents/{doc}/download", stream=True, headers=headers).content 
		output.write(bits)

doc = uploadDoc("Las Ventanas - 3819 Vineyard Ave, Pleasanton, CA 94566 _ Redfin.html", CLIENT_ID, CLIENT_SECRET)
print(f"Uploaded doc to Foxit, id is {doc['documentId']}")

task = htmlToPDF(doc["documentId"], CLIENT_ID, CLIENT_SECRET)
print(f"Created task, id is {task['taskId']}")
result = checkTask(task["taskId"], CLIENT_ID, CLIENT_SECRET)
print(f"Final result: {result}")

downloadResult(result["resultDocumentId"], "./fromhtml.pdf", CLIENT_ID, CLIENT_SECRET)
print("Done and saved to: ./fromhtml.pdf")

Uploaded doc to Foxit, id is 68b7d0b78fb8a36f7cb47607
Created task, id is 68b7d0b8d45ce57595c8c4e1
Current status, IN_PROGRESS, percentage: 10
Current status, IN_PROGRESS, percentage: 20
Final result: {'taskId': '68b7d0b8d45ce57595c8c4e1', 'status': 'COMPLETED', 'progress': 100, 'resultDocumentId': '68b7d0c2171e057d95265030'}
Done and saved to: ./fromhtml.pdf


# Foxit API compressor




In [5]:
def uploadDoc(path, id, secret):
	
	headers = {
		"client_id":id,
		"client_secret":secret
	}

	with open(path, 'rb') as f:
		files = {'file': f}

		request = requests.post(f"{HOST}/pdf-services/api/documents/upload", files=files, headers=headers)
		return request.json()


In [43]:
def compressPDF(doc, id, secret, level="HIGH"):
	
	headers = {
		"client_id":id,
		"client_secret":secret,
		"Content-Type":"application/json"
	}

	body = {
		"documentId":doc,
		"compressionLevel":level
	}

	request = requests.post(f"{HOST}/pdf-services/api/documents/modify/pdf-compress", json=body, headers=headers)
	return request.json()

In [44]:
def checkTask(task, id, secret):

	headers = {
		"client_id":id,
		"client_secret":secret,
		"Content-Type":"application/json"
	}

	done = False
	while done is False:
		request = requests.get(f"{HOST}/pdf-services/api/tasks/{task}", headers=headers)
		status = request.json()
		if status["status"] == "COMPLETED":
			done = True
			# really only need resultDocumentId, will address later
			return status
		elif status["status"] == "FAILED":
			print("Failure. Here is the last status:")
			print(status)
			sys.exit()
		else:
			print(f"Current status, {status['status']}, percentage: {status['progress']}")
			sleep(5)

In [56]:
def downloadResult(doc, path, id, secret):
	
	headers = {
		"client_id":id,
		"client_secret":secret
	}

	with open(path, "wb") as output:
		
		bits = requests.get(f"{HOST}/pdf-services/api/documents/{doc}/download", stream=True, headers=headers).content 
		output.write(bits)
doc = uploadDoc("fromhtml.pdf", CLIENT_ID, CLIENT_SECRET)
print(f"Uploaded pdf to Foxit, id is {doc['documentId']}")

Uploaded pdf to Foxit, id is 68b891406b586632cc543612


In [57]:
task = compressPDF(doc["documentId"], CLIENT_ID, CLIENT_SECRET)
print(f"Created task, id is {task['taskId']}")
result = checkTask(task["taskId"], CLIENT_ID, CLIENT_SECRET)
print(f"Final result: {result}")

Created task, id is 68b89145dedfd24189964ad8
Final result: {'taskId': '68b89145dedfd24189964ad8', 'status': 'COMPLETED', 'progress': 100, 'resultDocumentId': '68b89145866e93517dd4c20a'}


In [59]:
downloadResult(result["resultDocumentId"], "compressor.pdf", CLIENT_ID, CLIENT_SECRET)
print("compressor.pdf")

compressor.pdf


# Utilizing Foxit API to convert DOCX to PDF and add data into PDF

In [53]:
import os
import requests
import sys 
from time import sleep 
import base64 
from datetime import datetime
import json

base_url = "https://na1.fusion.foxit.com"
client_id = "foxit_QXn-i7EcDW1Uvgdh"
client_secret = "EebeG3bXf3XMui-9P0BA9oTiXxjs-dZs"

SAMPLE_DOCX_FILEPATH = "./docgen_sample.docx"
with open(SAMPLE_DOCX_FILEPATH, 'rb') as file:
	bd = file.read()
	b64 = base64.b64encode(bd).decode('utf-8')

data = {
	"name":"Raymond Camden", 
	"food": "sushi",
	"favoriteMovie": "Star Wars",
	"cats": [
		{"name":"Elise", "gender":"female", "age":14 },
		{"name":"Luna", "gender":"female", "age":13 },
		{"name":"Crackers", "gender":"male", "age":13 },
		{"name":"Gracie", "gender":"female", "age":12 },
		{"name":"Pig", "gender":"female", "age":10 },
		{"name":"Zelda", "gender":"female", "age":2 },
		{"name":"Wednesday", "gender":"female", "age":1 },
	],
}

payload = {
		"outputFormat":"pdf",
		"documentValues": data,  
		"base64FileString":b64
	}
body = json.dumps(payload)

headers = {
		"client_id":client_id,
		"client_secret":client_secret
	}
conn = http.client.HTTPSConnection("na1.fusion.foxit.com")
conn.request("POST", "/document-generation/api/GenerateDocumentBase64", body, headers)

resp = conn.getresponse()
reader = resp.read()
print(reader.decode("utf-8"))

{"message":"PDF Document Generated Successfully","fileExtension":"pdf","base64FileString":"..."}


In [55]:
parsed = json.loads(reader)
b64_str = parsed["base64FileString"]


In [57]:
binary_data = base64.b64decode(b64_str)

with open("output.pdf", "wb") as f:
    f.write(binary_data)